In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from mjd_path import *

np.random.seed(496)
tf.random.set_seed(496)

tf.keras.backend.set_floatx('float64')

directory = '.'

# network for synthetic data
activation = tf.tanh
hidden_layer = [20,20]
n_outputs = 1
learning_rate = 1e-5

#input processing
def process_input(X_input, X_input_):
	r = tf.fill([tf.shape(input=X_input)[0],1], np.float64(0.), name = 'r') # interest rate, if applicable

	S = tf.slice(X_input, (0,0), (-1,1))
	K = tf.slice(X_input, (0,1), (-1,1))
	T = tf.slice(X_input, (0,2), (-1,1))
	
	S_ = tf.slice(X_input_, (0,0), (-1,1))
	T_ = tf.slice(X_input_, (0,1), (-1,1))
	return S, K, T, S_, T_, r

# define neural network architecture
ann = tf.keras.Sequential(
    layers=[tf.keras.layers.Dense(hidden_layer[0], activation = activation, input_shape = (2,))] + \
	[tf.keras.layers.Dense(hidden_layer[i], activation = activation) for i in range(1, len(hidden_layer))] + \
	[tf.keras.layers.Dense(n_outputs, activation = tf.keras.activations.softplus)],
    name="ann")

# define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate)
# define loss function
hedging_mse = tf.keras.losses.MeanSquaredError()

#evaluate loss function and gradient
@tf.function
def loss(X_input, X_input_):
	S, K, T, S_, T_, r = process_input(X_input, X_input_)
	
	with tf.GradientTape() as tape1:
		tape1.watch(S)
		with tf.GradientTape() as tape2:
			tape2.watch([S, T])
			X = tf.concat([S/(K*tf.exp(-r*T)), T], 1) #input matrix for ANN
			X_ = tf.concat([S_/(K*tf.exp(-r*T_)), T_], 1) #input matrix for ANN_

			out = ann(X)
			out = K*tf.where(tf.greater(T, 1e-3), out, tf.maximum(S/K - 1, 0))
			out_ = ann(X_)
			out_ = K*tf.where(tf.greater(T_, 1e-3), out_, tf.maximum(S_/K - 1, 0))
		delta, theta = tape2.gradient(out, [S, T])
	gamma = tape1.gradient(delta, S)
	
	'''
	with tf.GradientTape() as tape:
		tape.watch([S, T])
		X = tf.concat([S/(K*tf.exp(-r*T)), T], 1) #input matrix for ANN
		X_ = tf.concat([S_/(K*tf.exp(-r*T_)), T_], 1) #input matrix for ANN_

		out = ann(X)
		out = K*tf.where(tf.greater(T, 1e-3), out, tf.maximum(S/K - 1, 0))
		out_ = ann(X_)
		out_ = K*tf.where(tf.greater(T_, 1e-3), out_, tf.maximum(S_/K - 1, 0))
	delta, theta = tape.gradient(out, [S, T])
	'''	
	return hedging_mse(delta*(S_-S) + 0.5 * gamma * tf.math.square(S_-S) + theta * (1/250), out_-out)
#	loss=hedging_mse(y_true=delta*(S_-S), y_pred=out_-out)
#	return loss

@tf.function
def grad(X_train, X_train_):
	with tf.GradientTape() as tape:
		tape.watch(ann.trainable_variables)
		loss_value = loss(X_train, X_train_)
	return loss_value, tape.gradient(loss_value, ann.trainable_variables)

# define training ops
@tf.function
def training_op(X_train, X_train_):
	loss_value, grads = grad(X_train, X_train_)
	optimizer.apply_gradients(zip(grads, ann.trainable_variables))

# Simulating geometric Brownian motion
def stock_sim_path(S, T, alpha, sigma, Lambda, a, b):
    mp = ModelParameters(all_s0=S,
                     all_r0=0,
                     all_time=T * 250,
                     all_delta=0.004, #0.00396825396, # daily
                     all_sigma=sigma,
                     gbm_mu=alpha,
                     jumps_lamda=Lambda, # lambda per day
                     jumps_sigma=b,
                     jumps_mu=a)
    return geometric_brownian_motion_jump_diffusion_levels(mp)

def get_batch2(stock_path,n, moneyness_range = (.5,2)): 
    """Constructs theoretical options based on the time series stock_path"""
    picks = np.random.randint(0, len(stock_path)-1, n)
    T = np.random.randint(1, 150, (n,1))
    S = stock_path[picks]
    S_ = stock_path[picks+1]
    K = np.random.uniform(*moneyness_range, (n,1))*S
    X = np.hstack([S, K, T/250])
    X_ = np.hstack([S_, (T-1)/250])
    return X, X_

#model training
n_epochs = 500 #number of training epochs
n_batches = 1000 #number of batches per epoch
batch_size = 10000 #number of theoretical options in each batch T = 2 #years of training data
T = 2 #years of training data
days = int(250*T)
stock_path = stock_sim_path(100, T, 0.1720, 0.1114, 64/252, -1.146e-3, 1.505e-2) #simulate stock path
stock_path_test = stock_sim_path(100, T, 0.1720, 0.1114, 64/252, -1.146e-3, 1.505e-2) #simulate stock path for cross-validation

#plot stock paths
#plt.plot(stock_path, label = 'Training')
#plt.plot(stock_path_test, label = 'Test')
#plt.legend()
#plt.show()

X_test, X_test_ = get_batch2(stock_path_test, batch_size) #get test-set


#TRAINING
losses = []
print("START TRAINING")
for epoch in range(n_epochs):
	for batch in range(n_batches):
		X_train, X_train_ = get_batch2(stock_path, batch_size) # get batch of theoretical options
		training_op(X_train, X_train_)
	epoch_loss = loss(X_test, X_test_)
	losses.append(epoch_loss)
	print('Epoch:', epoch, 'Loss:', epoch_loss) #, 'BS Loss:', bs_hedging_mse.eval({X_input: X_test, X_input_: X_test_}))
ann.save('model.h5')

plt.figure()
plt.semilogy(np.arange(n_epochs), losses)
plt.xlim([0, n_epochs])
plt.savefig('loss.png')


ModuleNotFoundError: No module named 'mjd_path'